In [ ]:


### THIS FUNCTION RETURNS THE PREDICTION FOR TRAIN and TEST IN STACKING
def Stacking(model,train,fold,y,test):
    folds=fold
    test_pred=np.empty((test.shape[0],1),float)
    train_pred=np.empty((0,1),float)
    t = 0
    for train_indices,val_indices in folds.split(train,y.values):
     
        x_train,x_val=train.iloc[train_indices],train.iloc[val_indices]
        y_train,y_val=y.iloc[train_indices],y.iloc[val_indices]

        model.fit(X=x_train,y=y_train)
        train_pred=np.append(train_pred,model.predict(x_val))
        print(model.score(x_val, y_val))
        test_pred=model.predict(test)
    return test_pred.reshape(-1,1),train_pred



lm_parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True]}

lm = GridSearchCV(LinearRegression(),
                                 param_grid=lm_parameters,
                                 cv=tscv,return_train_score=True)

test_pred1 ,train_pred1=Stacking(model=lm , train=xwe_Train,fold=tscv,test=xwe_Test,y=ywe_cnt_train)

train_pred1=pd.DataFrame(train_pred1)
test_pred1=pd.DataFrame(test_pred1)

RF_parameters = {'n_estimators': [10, 30 ,100],
                                             'bootstrap': [True],
                                             'max_depth': [80, 100 ],
                                             'max_features': ['sqrt',16 ,32],
                                             'min_samples_leaf': [2,  5 , 8],
                                             'min_samples_split': [ 10 , 8 , 15],
                                            'random_state':[random_seed],
                                            'criterion':['mse']}
rf = GridSearchCV(RandomForestRegressor(),
                                 param_grid= RF_parameters,
                                 cv=tscv)

test_pred2 ,train_pred2=Stacking(model=rf ,train=xwe_Train,fold=tscv,test=xwe_Test,y=ywe_cnt_train)

train_pred2=pd.DataFrame(train_pred2)
test_pred2=pd.DataFrame(test_pred2)

param_grid = {'learning_rate': [0.01, 0.15  ,0.2], 
          'max_depth': [8,12 , 16],
          'min_child_weight': [2, 3,5,10],
          'subsample': [0.5,  0.8],
          'colsample_bytree': [0.5, 0.75],
          'n_estimators': [18, 50 , 100],
}

model = xgb.XGBRegressor()

xg1 = GridSearchCV(model,
                    param_grid = param_grid,
                    cv = tscv,
                    n_jobs = -1,
                    scoring = 'r2',
                    verbose=True)

test_pred3 ,train_pred3=Stacking(model=xg1 ,train=xwe_Train,fold=tscv,test=xwe_Test,y=ywe_cnt_train)

train_pred3=pd.DataFrame(train_pred3)
test_pred3=pd.DataFrame(test_pred3)


##### if doing time series fold check teh number of rows that where skipped
df = pd.concat([train_pred1, train_pred2, train_pred3,xwe_Train.drop(np.arange(0,795)).reset_index(drop=True) ], axis=1)
df_test = pd.concat([test_pred1, test_pred2,test_pred3,xwe_Test.reset_index(drop=True)], axis=1)

## Or any model 

param_grid = {'learning_rate': [0.01, 0.15  ,0.2], 
          'max_depth': [8,12 , 16],
          'min_child_weight': [2, 3,5,10],
          'subsample': [0.5,  0.8],
          'colsample_bytree': [0.5, 0.75],
          'n_estimators': [18, 50 , 100],
}

model = xgb.XGBRegressor()

xg1 = GridSearchCV(model,
                    param_grid = param_grid,
                    cv = tscv,
                    n_jobs = -1,
                    scoring = 'r2',
                    verbose=True)

test_pred3 ,train_pred3=Stacking(model=xg1 ,train=xwe_Train,fold=tscv,test=xwe_Test,y=ywe_cnt_train)

train_pred3=pd.DataFrame(train_pred3)
test_pred3=pd.DataFrame(test_pred3)